In [1]:
#Importando librerias
import wget
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

#Definiendo las variables globales
global df_traffic, resultados, modelo, modelo_clasificacion

In [2]:
#Extraemos el contenido del DataFrame y aplicamos la función preprocesamiento y tomamos una muestra para verificar las columnas desplegadas
df_traffic = pd.read_csv('https://raw.githubusercontent.com/ElProfeAlejo/Bootcamp_Databases/main/traffic_site.csv', dtype={'date':object,'fullVisitorId':object,'visitId':object})
df_traffic.sample(5)

,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
9361,Direct,20161024,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",603373162136570069,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",603373162136570069_1477318648,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",1477318648,22,1477318648
5261,Organic Search,20161102,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",9197292506380217506,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",9197292506380217506_1478078246,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""23"", ""pageviews"": ""18...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1478078246,1,1478078246
3722,Social,20170112,"{""browser"": ""Android Webview"", ""browserVersion...",9934552462496041634,"{""continent"": ""Europe"", ""subContinent"": ""Weste...",9934552462496041634_1484252952,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1484252952,1,1484252952
12043,Organic Search,20160907,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",384821792030772632,"{""continent"": ""Asia"", ""subContinent"": ""Souther...",0384821792030772632_1473262321,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""18"", ""pageviews"": ""15...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1473262321,1,1473262321
9368,Referral,20161010,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3830234049092232846,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",3830234049092232846_1476134975,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""12"", ""pageviews"": ""9""}","{""referralPath"": ""/"", ""campaign"": ""(not set)"",...",1476134975,2,1476134975


In [3]:
#Verificamos el número de columnas y filas contenidas en el DataFrame y su tipo de dato asignado a cada una
df_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12283 entries, 0 to 12282
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   channelGrouping       12283 non-null  object
 1   date                  12283 non-null  object
 2   device                12283 non-null  object
 3   fullVisitorId         12283 non-null  object
 4   geoNetwork            12283 non-null  object
 5   sessionId             12283 non-null  object
 6   socialEngagementType  12283 non-null  object
 7   totals                12283 non-null  object
 8   trafficSource         12283 non-null  object
 9   visitId               12283 non-null  object
 10  visitNumber           12283 non-null  int64 
 11  visitStartTime        12283 non-null  int64 
dtypes: int64(2), object(10)
memory usage: 1.1+ MB


In [ ]:
# Verificamos el tipo de dato contenido en las columnas anidadas
df_traffic.device.value_counts().index

In [ ]:
def preprocesamiento():
    global df_traffic

    # Extraer datos de diccionarios
    diccionarios = ['device','geoNetwork','trafficSource','totals']
    for columna in diccionarios:
        df_traffic = df_traffic.join(pd.DataFrame([json.loads(linea) for linea in df_traffic[columna]]))
    df_traffic = df_traffic.drop(columns=diccionarios)
    # o puedo ustilizar esta otra opción df_traffic.drop(columns=diccionarios, inplace=True)

    #Eliminar las columnas que no requerimos
    df_traffic= df_traffic.drop(columns=['socialEngagementType','browserVersion','browserSize','operatingSystemVersion','mobileDeviceBranding','mobileDeviceModel','mobileInputSelector','mobileDeviceInfo','mobileDeviceMarketingName','flashVersion','language','screenColors','screenResolution','continent','subContinent','country','region','cityId','latitude','longitude','networkLocation','keyword','adwordsClickInfo','isTrueDirect','adContent','campaignCode','networkDomain','visits'])

    # Eliminar duplicados
    df_traffic = df_traffic.drop_duplicates()

    # Reemplazar textos por números
    cuant = ['bounces', 'hits','newVisits','pageviews', 'transactionRevenue']
    for columna in cuant:
        df_traffic[columna] = pd.to_numeric(df_traffic[columna])
    df_traffic[cuant] = df_traffic[cuant].fillna(0)
    df_traffic['transactionRevenue'] = df_traffic['transactionRevenue'] / 1000000


   # Reemplazar números por fechas
    df_traffic['date'] = pd.to_datetime(df_traffic['date'], format='%Y%m%d')
    df_traffic['visitStartTime'] = pd.to_datetime(df_traffic['visitStartTime'], unit='s')
    df_traffic.bounces = df_traffic.bounces.astype(int)
    df_traffic.newVisits = df_traffic.newVisits.astype(int)



preprocesamiento()
df_traffic.sample(5)

In [ ]:
#Verificamos el número de columnas y filas contenidas en el DataFrame y su tipo de dato asignado a cada una
df_traffic.info()

#**3. Exploración y Feature Engineering**


---

In [ ]:
#Agregamos nuevas columnas a partir de las columnas "date" y "visitStartTime" para tener más información capaz de resolver preguntas con base en los datos
def feature_engineering():
    global df_traffic, resultados, modelo, modelo_clasificacion
    
    # Crear las nuevas columnas
    df_traffic['year'] = df_traffic['date'].apply(lambda x:x.year)
    df_traffic['month'] = df_traffic['date'].dt.month  # Mes
    df_traffic['quarterMonth'] = df_traffic['date'].dt.quarter  # Trimestre del mes
    df_traffic['day'] = df_traffic['date'].dt.day  # Día
    df_traffic['weekday'] = df_traffic['date'].dt.weekday  # Día de la semana (0: Lunes, 1: Martes, ..., 6: Domingo)
    
    df_traffic["hour"] = df_traffic["visitStartTime"].dt.hour

feature_engineering()   

In [ ]:
#Realizamos la consulta de número de columnas
df_traffic.info()

#**3.5 Analisis Univariable de las variables categóricas del Dataset**


---


Tablas de distribución de frecuencias

A través de la librería funpymodeling a través de su función freq_tbl, realizará análisis univariable de las variables categóricas del dataset para determinar en cada fila su frecuencia, su representanción porcentaje y porcentaje acumulado, para usar esta función únicamente es necesario agregar el nombre del DataFrame y de manera automática dará la relación de las variables CATEGÓRICAS, sin embargo. también puede accederse para realizarlo a través de la subconsulta de la columna específica a consultar. 

Ejemplo : freq_tbl(df["Nombre_columna"])

In [ ]:
#Función utilizada para el analisis univariable de las columnas con datos categóricos
freq_tbl(df_traffic)

In [ ]:
#Función para arrojar datos de anpalisis de la variable 
def analyze_city_data(df, columna):
    
    # Mostrar el conteo de los 10 valores que MÁS se repiten en la información de las primeras filas de la columna asignada
    print("\nConteo de los 10 valores que MÁS se repiten en las primeras filas de la columna asignada:")
    print(df[columna].value_counts().head(10))
    

    # Mostrar los primeros valores en orden descendente de la columna asignada
    print("\nPrimeros valores de la columna asignada")
    print(df[columna].head(10))



    # Mostrar estadísticas descriptivas de la columna asignada
    print("\nEstadísticas descriptivas de la columna asignada")
    print(df[columna].describe())

# Llamar a la función con el DataFrame df_traffic
analyze_city_data(df_traffic, "city")

In [ ]:
def buscar_repetidos(df, columna, valor_frecuencia):
    # Contar la frecuencia de cada dato dentro de las filas de la columna especificada
    frecuencia_datos = df[columna].value_counts()

    # Filtrar los datos que tienen una frecuencia igual al valor especificado
    datos_frecuencia_var = frecuencia_datos[frecuencia_datos == valor_frecuencia]

    # Mostrar los datos que aparecen con la frecuencia especificada
    print("Datos con frecuencia de", valor_frecuencia, "en la columna", columna, ":")
    print(datos_frecuencia_var)

# Llamar a la función con los parámetros adecuados
buscar_repetidos(df_traffic, "city", 10) # Cambia "city" por la columna deseada y 10 por el valor de frecuencia buscado

1.- df_traffic["channelGrouping"] # Aporta información valiosa entre 7 categorías, sin Datos Nulos

2.- df_traffic["fullVisitorId"] # Aporta en un porcentaje pequeño información relevante
De la columna frequency, 8780 de los 12283 registros cuentan con frecuencia de 1 y 784 con frecuencia de 2, lo que da un porcentaje acumulado de los primeros 2 valores del 77.92%. Por lo tanto, fuera de esas dos opciones, la cantidad de información para trabajar con el modelo termina siendo muy pequeña. Sin embargo, se conservará la columna.

3.- df_traffic["sessionId"] # Eliminado debido a que el esquema de BigQuery lo determina como obsoleto y recomienda usar fullVisitorId en su lugar.

4.- df_traffic["visitId"] # Eliminado debido a que solo 11 valores de los 12,272 registros tienen una frecuencia de 2, todos los demás tienen una frecuencia de 1, por lo tanto, al ser tan poca la información para comparar, se elimina la columna.

5.- df_traffic["browser"] # Aporta información valiosa con una distribución de más del 90% de los datos en las primeras 3 categorías.

6.- df_traffic["operatingSystem"] # Aporta información valiosa con una distribución de más del 90% de los datos en las primeras 4 categorías.

7.- df_traffic["deviceCategory"] # Aporta información importante distribuida en 3 categorías.

8.- df_traffic["metro"] # El 77.7 % son datos nulos, sin embargo, se conservarán los pocos datos al ser un área de mercado designada desde la que se originan las sesiones.

9.- df_traffic["city"] # Variable que aporta la ciudad de los usuarios, extraída de las direcciones IP o de los IDs geográficos. Sin embargo, cuenta con más de un 50% de datos nulos. Se conserva.

10.- df_traffic["campaign"] # Variable con 95% de valores no establecidos. A consideración de borrarla, se probará su peso en el modelo.

11.- df_traffic["source"] # Aporta información valiosa. El 93% de los datos dentro de sus primeras 5 categorías.

12.- df_traffic["referralPath"] # Verificamos la información aportada por la columna. En este caso, la columna cuenta con menos de 200 registros de las 12276 filas, de los cuales ninguno aporta información importante. Por lo mismo, se decide ser eliminado del modelo.

13.- df_traffic["medium"] # Cuenta con un 16% de nulos. Sin embargo, aporta información valiosa sobre el medio de la fuente de tráfico.


Explicación parametros de Fuente de Trafico [medium]

Tráfico orgánico ("organic"): Este tipo de tráfico proviene de los resultados de búsqueda orgánica en los motores de búsqueda como Google, Bing, Yahoo, entre otros. Es decir, cuando los usuarios encuentran y visitan el sitio web haciendo clic en un enlace que aparece en los resultados de búsqueda sin haber pagado por la posición.

Coste por clic ("cpc"): También conocido como tráfico de pago por clic, se refiere al tráfico que llega al sitio web a través de anuncios de pago en motores de búsqueda, redes de anuncios o plataformas de publicidad en línea. Los anunciantes pagan cada vez que un usuario hace clic en su anuncio y es dirigido al sitio web.

Referencia ("referral"): Este tipo de tráfico proviene de enlaces en otros sitios web diferentes a los motores de búsqueda. Por ejemplo, si otro sitio web incluye un enlace al sitio web en su contenido y los usuarios hacen clic en ese enlace para visitar el sitio, se considera tráfico de referencia.

In [ ]:
#Se eliminan las 3 columnas que menos aportan información al modelo de acuerdo al analisis univariable anterior
df_traffic.drop(columns=['referralPath', "sessionId", "visitId"], inplace=True)